In [1]:
import cv2
import numpy as np
import tensorflow as tf

2026-02-20 16:53:11.783117: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-20 16:53:11.783380: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-20 16:53:11.815244: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-20 16:53:12.491465: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [2]:
detections_global = []
selected_box = None
tracking = False
tracker = None

In [3]:
interpreter = tf.lite.Interpreter(model_path="detect.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

/home/premesh/vs_code/object_focus/.venv/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
labels = []
with open("labelmap.txt","r") as f:
    for line in f:
        labels.append(line.strip())

In [7]:
import cv2
import numpy as np
import tensorflow as tf

detections_global = []
selected_box = None
tracking = False
tracker = None
current_frame = None

interpreter = tf.lite.Interpreter(model_path="detect.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

height = input_details[0]['shape'][1]
width  = input_details[0]['shape'][2]

labels = []
with open("labelmap.txt", "r") as f:
    for line in f:
        labels.append(line.strip())

def compute_iou(boxA, boxB):
    ax1, ay1, ax2, ay2 = boxA
    bx1, by1, bx2, by2 = boxB

    inter_x1 = max(ax1, bx1)
    inter_y1 = max(ay1, by1)
    inter_x2 = min(ax2, bx2)
    inter_y2 = min(ay2, by2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    area_a = (ax2 - ax1) * (ay2 - ay1)
    area_b = (bx2 - bx1) * (by2 - by1)
    union = area_a + area_b - inter_area

    return inter_area / union if union > 0 else 0


def mouse_click(event, x, y, flags, param):
    global selected_box, tracking, tracker

    if event == cv2.EVENT_LBUTTONDOWN:
        for box in detections_global:
            x1, y1, x2, y2, label = box
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_box = (x1, y1, x2, y2)
                tracking = True
                tracker = cv2.legacy.TrackerKCF_create()
                tracker.init(current_frame, (x1, y1, x2 - x1, y2 - y1))
                print(f"Now tracking: {label}")
                break


cap = cv2.VideoCapture(0)
cv2.namedWindow("Object Detection")
cv2.setMouseCallback("Object Detection", mouse_click)

while True:
    ret, frame = cap.read()
    if not ret:                      
        break

    current_frame = frame.copy()
    detections_global = []           

    h, w, _ = frame.shape

    img = cv2.resize(frame, (width, height))
    input_data = np.expand_dims(img, axis=0).astype(np.uint8)

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    boxes   = interpreter.get_tensor(output_details[0]['index'])[0]
    classes = interpreter.get_tensor(output_details[1]['index'])[0]
    scores  = interpreter.get_tensor(output_details[2]['index'])[0]

    for i in range(len(scores)):
        if scores[i] > 0.5:
            ymin, xmin, ymax, xmax = boxes[i]

            x1 = int(xmin * w)
            y1 = int(ymin * h)
            x2 = int(xmax * w)
            y2 = int(ymax * h)

            label = labels[int(classes[i])]
            detections_global.append((x1, y1, x2, y2, label))

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            if label != '???':
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    if tracking and tracker is not None:
        success, bbox = tracker.update(frame)

        if success:
            tx, ty, tw, th = [int(v) for v in bbox]
            selected_box = (tx, ty, tx + tw, ty + th)

            cv2.rectangle(frame, (tx, ty), (tx + tw, ty + th), (0, 245, 160), 3)
            cv2.putText(frame, "TRACKING", (tx, ty - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 245, 160), 2)
        else:
            best_iou = 0
            best_box = None
            for box in detections_global:
                x1, y1, x2, y2, label = box
                score = compute_iou(selected_box, (x1, y1, x2, y2))
                if score > best_iou:
                    best_iou = score
                    best_box = (x1, y1, x2, y2)

            if best_iou > 0.1 and best_box:
                bx1, by1, bx2, by2 = best_box
                tracker = cv2.TrackerCSRT_create()
                tracker.init(frame, (bx1, by1, bx2 - bx1, by2 - by1))

    cv2.imshow("Object Detection", frame)   
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        tracking = False
        tracker = None
        selected_box = None

cap.release()
cv2.destroyAllWindows()

Now tracking: ???
Now tracking: ???
Now tracking: hair drier
Now tracking: ???
Now tracking: ???
Now tracking: mouse
Now tracking: ???
Now tracking: ???
Now tracking: ???
Now tracking: ???


KeyboardInterrupt: 